In [217]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [218]:
%autoreload

import warnings
import os.path
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import geopy
import xgboost as xgb
import os
import shutil
import geopandas as gpd
import catboost as cb

from xgboost import XGBRegressor, plot_importance, to_graphviz, plot_tree
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold
from k_fold import random_k_fold
from shapely import wkt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from utils import squared_log, rmsle_xgb, add_city_centre_dist, group_ages, to_categorical, nan_to_string, object_encoder
from k_fold import random_k_fold, _rmsle, xgb_cross_validation, rmsle_scorer, RmsleMetric, RmsleObjective
from scipy.stats import uniform, randint

pd.options.mode.chained_assignment = None  # default='warn'

spatial = pd.read_csv('data/grunnkrets_norway_stripped.csv')
age = pd.read_csv('data/grunnkrets_age_distribution.csv')
income = pd.read_csv('data/grunnkrets_income_households.csv').set_index(['grunnkrets_id', 'year']).add_prefix('income_').reset_index()
households = pd.read_csv('data/grunnkrets_households_num_persons.csv')
submission = pd.read_csv('data/sample_submission.csv')
plaace = pd.read_csv('data/plaace_hierarchy.csv')
busstops = pd.read_csv('data/busstops_norway.csv')

train = pd.read_csv('data/stores_train.csv')
test = pd.read_csv('data/stores_test.csv')

submission = pd.read_csv('data/sample_submission.csv')
model_name = "modeling/0002.model"


In [219]:
def generate_features(df: pd.DataFrame, predictor: str = ''):
    age_ranges = [
        (0, 19),
        (20, 39),
        (40, 59),
        (60, 79),
        (80, 90),
    ]
    
    # Define datasets to be merged
    spatial_merge = spatial.drop(columns=['year']).drop_duplicates(subset=['grunnkrets_id'])
    age_groups_merge = group_ages(age, age_ranges)
    income_merge = income.drop(columns=['year']).drop_duplicates(subset='grunnkrets_id')
    households_merge = households.drop(columns=['year']).drop_duplicates(subset='grunnkrets_id')
    plaace_merge = plaace.drop_duplicates(subset='plaace_hierarchy_id')
    bus_data_train_merge = gpd.read_parquet('derived_data/stores_bus_stops_lt_1km_train').drop(columns=['geometry'])

    # Merge datasets
    df = df.merge(spatial_merge, on='grunnkrets_id', how='left')
    df = df.merge(age_groups_merge, on='grunnkrets_id', how='left')
    df = df.merge(income_merge, on='grunnkrets_id', how='left')
    df = df.merge(households_merge, on='grunnkrets_id', how='left')
    df = df.merge(plaace_merge, how='left')
    df = df.merge(bus_data_train_merge, on='store_id', how='left')
    df = add_city_centre_dist(df).drop(columns=['lon_center', 'lat_center'])

    # Handle categories for different predictors
    if predictor == 'xgb':
        # df = to_categorical(df)
        df = object_encoder(df)
    elif predictor == 'catboost':
        df = nan_to_string(df)
    else: 
        raise ValueError('Invalid predictor')

    features = [
        'store_name', 
        'mall_name', 
        'chain_name',
        'address', 
        'lat', 'lon',
        
        *age_groups_merge.drop(columns=['grunnkrets_id']).columns,
        *income_merge.drop(columns=['grunnkrets_id']).columns,
        *households_merge.drop(columns=['grunnkrets_id']).columns,
        'lv1_desc', 'lv2_desc',
        *bus_data_train_merge.drop(columns=['store_id']).columns,
        'dist_to_center'
    ]

    return df[features]


label_name = 'revenue'
X = train.drop(columns=[label_name])
y = train[label_name]


In [220]:
def plot_corr(data):
  df = data[['revenue', 
    # 'age_0_19', 'age_20_39', 'age_40_59', 'age_60_79', 'age_80_90', 
    # 'bus_stops_count', 'Mangler viktighetsnivå', 'Standard holdeplass', 'Lokalt knutepunkt', 'Nasjonalt knutepunkt', 'Regionalt knutepunkt', 'Annen viktig holdeplass', 
    'dist_to_center', 'lat','lon'
    ]]
  df['knutepunkt'] = data[['Lokalt knutepunkt', 'Nasjonalt knutepunkt', 'Regionalt knutepunkt']].sum(axis=1)
  # df.revenue = np.exp(df.revenue)
  # df.bus_stops_count = np.sqrt(df.bus_stops_count)
  df = df[df.dist_to_center < 70_000]
  # df.dist_to_center = np.log(df.dist_to_center)
  
  plt.figure(figsize=(15, 15))
  pairplot = sns.pairplot(df)
  # heatmap = sns.heatmap(df.corr(), vmin=-1, vmax=1, annot=True)


# data_full =  pd.merge(X_train, y_train, left_index=True, right_index=True) 
# plot_corr(data_full)


In [221]:
def clear_buffers(X_train, y_train, X_val, y_val):
    # Clear buffers
    folder = os.path.join(os.getcwd(), 'modeling')

    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        if os.path.isfile(file_path):
            os.unlink(file_path)
            print(f'Deleted file: {file_path}')

    train_buffer_path = 'modeling/train.buffer'
    test_buffer_path = 'modeling/test.buffer'

    dtrain = xgb.DMatrix(data=X_train, label=y_train, enable_categorical=True)
    dtrain.save_binary(train_buffer_path)
    print(f'--> {train_buffer_path} created and saved.')

    dvalid = xgb.DMatrix(data=X_val, label=y_val, enable_categorical=True)
    dvalid.save_binary(test_buffer_path)
    print(f'--> {test_buffer_path} created and saved.')

    return dtrain, dvalid

In [222]:
# print(model.best_score_)
# y_pred_train = model.predict(X_train)
# y_pred_val = model.predict(X_val)
# print(_rmsle(y_train, y_pred_train))
# print(_rmsle(y_val, y_pred_val))

In [223]:
def train_xgb_model(X_train, y_train, X_val, y_val):
    params = {'colsample_bytree': 0.7717138210314867, 'learning_rate': 0.047506668950627134, 'max_depth': 8, 'min_child_weight': 3, 'n_estimators': 223, 'subsample': 0.9929036803032936}
    print('Clearing and creating buffers...')
    dtrain, dvalid = clear_buffers(X_train, y_train, X_val, y_val)
    
    rand_search_model = random_k_fold(X_train, y_train, verbose=1, n_iter=100)
    model = rand_search_model
    params = model.best_params_
    print(rand_search_model.best_score_, params)
    
    # params = {'colsample_bytree': 0.8601277878899238, 'eval_metric': 'rmsle', 'gamma': 0.12760202929262826, 'learning_rate': 0.07356461924449906, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 306, 'objective': 'reg:squaredlogerror', 'subsample': 0.8993341396761092}
    
    params['disable_default_eval_metric'] = True
    # model = XGBRegressor()
    # model.set_params(**params)
    # model.fit(X_train, y_train)
    # y_pred_train = model.predict(X_train)
    # y_pred_val = model.predict(X_val)
    # print(_rmsle(y_train, y_pred_train))
    # print(_rmsle(y_val, y_pred_val))

    # num_round = 999
    # watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

    # print("Attempting to start training...")
    # model = xgb.train(
    #     params=params, 
    #     dtrain=dtrain, 
    #     num_boost_round=num_round, 
    #     evals=watchlist, 
    #     early_stopping_rounds=10, 
    #     verbose_eval=20)
    # print("--> model trained.")
    # print('Best score:', model.best_score)

    # print("Attempting to save model...")
    # model.save_model(model_name)
    # print("--> model saved.")

    return model


# X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=.8)
# X_train, X_val = generate_features(X_train, predictor='xgb'), generate_features(X_val, predictor='xgb')

# model = train_xgb_model(X_train, y_train, X_val, y_val)

In [224]:
# y_pred_train = model.predict(X_train)
# y_pred_val = model.predict(X_val)
# print(_rmsle(y_train, y_pred_train))
# print(_rmsle(y_val, y_pred_val))

In [225]:
def xgb_prediction(X_test, model):
    dtest = xgb.DMatrix(data=X_test, enable_categorical=True)

    print("\nAttempting to start prediction...")
    y_pred = model.predict(dtest, ntree_limit=model.best_iteration)
    print("--> Prediction finished.")

    print("\nAttempting to save prediction...")
    submission['predicted'] = np.array(y_pred)
    submission.to_csv('submissions/submission.csv', index=False)
    print("--> prediction saved with features as name in submission folder.")


# X_test = generate_features(test, predictor='xgb')
# xgb_prediction(X_test, model)

In [226]:
# xgb_model = model.best_estimator_ if model.best_estimator_ is not None else model
# xgb_model = model
# plot_importance(xgb_model)
# xgb.to_graphviz(xgb_model, num_trees=1)

### Prepare features for Catboost predictor

In [ ]:
def print_cv_summary(cv_data):
    cv_data.head(10)

    best_value = cv_data['test-Logloss-mean'].min()
    best_iter = cv_data['test-Logloss-mean'].values.argmin()

    print('Best validation Logloss score : {:.4f}±{:.4f} on step {}'.format(
        best_value,
        cv_data['test-Logloss-std'][best_iter],
        best_iter)
    )

param_dist = {
    'learning_rate': uniform(0.01, 0.3),
    'max_depth': randint(3, 9),
    'n_estimators': randint(100, 300),
    'objective': RmsleObjective(),
    'eval_metric': RmsleMetric()
}

cbr = cb.CatBoostRegressor()

rscv = RandomizedSearchCV(cbr, param_dist)

rscv.fit(X)

In [233]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=.8)
X_train, X_val = generate_features(X_train, predictor='catboost'), generate_features(X_val, predictor='catboost')
X_test = generate_features(test, predictor='catboost')

auxillary_columns = ['address']
text_features = ['store_name', 'address']
cat_features = ['mall_name', 'chain_name', 'lv1_desc', 'lv2_desc']

train_pool = cb.Pool(
    X_train,
    y_train,
    cat_features=cat_features,
    text_features=text_features,
    feature_names=list(X_train)
)

valid_pool = cb.Pool(
    X_val,
    y_val,
    cat_features=cat_features,
    text_features=text_features,
    feature_names=list(X_train)
)

params = {
    'iterations': 2000,
    'learning_rate': 0.03,
    'objective': RmsleObjective(),
    'eval_metric': RmsleMetric()
}

model = cb.CatBoostRegressor(**params)
model.fit(train_pool, eval_set=valid_pool, verbose=100)

y_pred = model.predict(X_test)
submission['predicted'] = np.array(y_pred)
submission.to_csv('submissions/submission.csv', index=False)


/home/koholm/Desktop/maskinprosjekt/venv/lib/python3.8/site-packages/catboost/core.py:2266: UserWarning: Failed to import numba for optimizing custom metrics and objectives
  _check_train_params(params)


0:	learn: 1.8492386	test: 1.8453068	best: 1.8453068 (0)	total: 129ms	remaining: 12.7s
10:	learn: 1.5638131	test: 1.5603391	best: 1.5603391 (10)	total: 1.44s	remaining: 11.7s
20:	learn: 1.3986032	test: 1.3962216	best: 1.3962216 (20)	total: 2.76s	remaining: 10.4s
30:	learn: 1.2901400	test: 1.2885741	best: 1.2885741 (30)	total: 4.11s	remaining: 9.14s
40:	learn: 1.2123795	test: 1.2115341	best: 1.2115341 (40)	total: 5.43s	remaining: 7.81s
50:	learn: 1.1544169	test: 1.1536003	best: 1.1536003 (50)	total: 6.76s	remaining: 6.5s
60:	learn: 1.1086333	test: 1.1081833	best: 1.1081833 (60)	total: 8.1s	remaining: 5.18s
70:	learn: 1.0723715	test: 1.0724645	best: 1.0724645 (70)	total: 9.42s	remaining: 3.85s
80:	learn: 1.0436111	test: 1.0440077	best: 1.0440077 (80)	total: 10.7s	remaining: 2.52s
90:	learn: 1.0198672	test: 1.0205508	best: 1.0205508 (90)	total: 12.1s	remaining: 1.19s
99:	learn: 1.0019782	test: 1.0029025	best: 1.0029025 (99)	total: 13.2s	remaining: 0us

bestTest = 1.002902546
bestIteration 

In [230]:
# y_pred = model.predict(test)
# y_pred

model.best_score_

{}

In [228]:
params = {
    'depth': randint(2, 20),
    'learning_rate': uniform(0.01, 0.4),
    'iterations': randint(10, 1000)
}

model = cb.CatBoostRegressor(loss_function=RmsleObjective, eval_metric=RmsleMetric)


# model.randomized_search(train_pool, param_distributions=params, cv=5)

pred = model.predict(X_val)
rmsle = (np.sqrt(mean_squared_log_error(y_val, pred)))
print('Testing performance')
print('RMSLE: {:.2f}'.format(rmsle))

CatBoostError: There is no trained model to use predict(). Use fit() to train model. Then use this method.